In [32]:
import datetime as dt
import time
import logging

from optibook.synchronous_client import Exchange
from optibook.common_types import InstrumentType, OptionKind

from math import floor, ceil, exp
from black_scholes import call_value, put_value, call_delta, put_delta
from libs import calculate_current_time_to_date

exchange = Exchange()
exchange.connect()

logging.getLogger('client').setLevel('ERROR')

2024-02-08 22:28:45,439 [asyncio   ] [MainThread  ] Using selector: EpollSelector
2024-02-08 22:28:45,478 [client    ] [Thread-7 (_thread_entry_point)] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.


In [33]:
def update_quotes(instrument_id, theoretical_price, credit, volume, position_limit, tick_size):
    """
    This function updates the quotes specified by <option_id>. We take the following actions in sequence:
        - pull (remove) any current oustanding orders
        - add credit to theoretical price and round to nearest tick size to create a set of bid/ask quotes
        - calculate max volumes to insert as to not pass the position_limit
        - reinsert limit orders on those levels

    Arguments:
        option_id: str           -  Exchange Instrument ID of the option to trade
        theoretical_price: float -  Price to quote around
        credit: float            -  Difference to subtract from/add to theoretical price to come to final bid/ask price
        volume:                  -  Volume (# lots) of the inserted orders (given they do not breach position limits)
        position_limit: int      -  Position limit (long/short) to avoid crossing
        tick_size: float         -  Tick size of the quoted instrument
    """

    # Print any new trades
    trades = exchange.poll_new_trades(instrument_id=instrument_id)

    # Pull (remove) all existing outstanding orders
    orders = exchange.get_outstanding_orders(instrument_id=instrument_id)
    for order_id, order in orders.items():
        exchange.delete_order(instrument_id=instrument_id, order_id=order_id)

    # Calculate bid and ask price
    bid_price = round_down_to_tick(theoretical_price - credit, tick_size)
    ask_price = round_up_to_tick(theoretical_price + credit, tick_size)

    # Calculate bid and ask volumes, taking into account the provided position_limit
    position = exchange.get_positions()[instrument_id]

    max_volume_to_buy = position_limit - position
    max_volume_to_sell = position_limit + position

    bid_volume = min(volume, max_volume_to_buy)
    ask_volume = min(volume, max_volume_to_sell)

    # Insert new limit orders
    if bid_volume > 0:
        exchange.insert_order(
            instrument_id=instrument_id,
            price=bid_price,
            volume=bid_volume,
            side='bid',
            order_type='limit',
        )
    if ask_volume > 0:
        exchange.insert_order(
            instrument_id=instrument_id,
            price=ask_price,
            volume=ask_volume,
            side='ask',
            order_type='limit',
        )

In [34]:
def get_CSCO_value(side):
    """
    Calculates the value of CSCO based on OB5X_ETF
    """
    M = 0.25
    C = 2.5
    val = 0 
    if(side=='ask'):
        if(len(exchange.get_last_price_book('NVDA').asks)==0 or len(exchange.get_last_price_book('ING').asks)==0 or len(exchange.get_last_price_book('SAN').asks)==0 or len(exchange.get_last_price_book('PFE').asks)==0 or len(exchange.get_last_price_book("OB5X_ETF").asks) == 0):
            return None
        NVDA = exchange.get_last_price_book(instrument_id='NVDA').asks[0].price
        ING = exchange.get_last_price_book(instrument_id='ING').asks[0].price
        SAN = exchange.get_last_price_book(instrument_id='SAN').asks[0].price
        PFE = exchange.get_last_price_book(instrument_id='PFE').asks[0].price
        OB5X_ETF = exchange.get_last_price_book(instrument_id='OB5X_ETF').asks[0].price

        val = (OB5X_ETF-C-M*(908.06*NVDA + 129.24*ING + 124.78*SAN + 2245.39*PFE)/1000)*1000/(M*953.21)
    else:
        if(len(exchange.get_last_price_book('NVDA').bids)==0 or len(exchange.get_last_price_book('ING').bids)==0 or len(exchange.get_last_price_book('SAN').bids)==0 or len(exchange.get_last_price_book('PFE').bids)==0 or len(exchange.get_last_price_book('CSCO').bids) == 0):
            return None
        NVDA = exchange.get_last_price_book(instrument_id='NVDA').bids[0].price
        ING = exchange.get_last_price_book(instrument_id='ING').bids[0].price
        SAN = exchange.get_last_price_book(instrument_id='SAN').bids[0].price
        PFE = exchange.get_last_price_book(instrument_id='PFE').bids[0].price
        OB5X_ETF = exchange.get_last_price_book(instrument_id='OB5X_ETF').bids[0].price

        val = (OB5X_ETF-C-M*(908.06*NVDA + 129.24*ING + 124.78*SAN + 2245.39*PFE)/1000)*1000/(M*953.21)

    return val

In [35]:
def round_down_to_tick(price, tick_size):
    """
    Rounds a price down to the nearest tick, e.g. if the tick size is 0.10, a price of 0.97 will get rounded to 0.90.
    """
    return floor(price / tick_size) * tick_size


def round_up_to_tick(price, tick_size):
    """
    Rounds a price up to the nearest tick, e.g. if the tick size is 0.10, a price of 1.34 will get rounded to 1.40.
    """
    return ceil(price / tick_size) * tick_size


def load_futures_for_underlying(underlying_group_id):

    all_instruments = exchange.get_instruments()

    futures = {instrument_id: instrument
               for instrument_id, instrument in all_instruments.items()
               if (instrument.instrument_type == InstrumentType.INDEX_FUTURE or instrument.instrument_type == InstrumentType.STOCK_FUTURE)
               and instrument.instrument_group == underlying_group_id}
    
    return futures
def load_instruments_for_underlying(underlying_stock_id):
    all_instruments = exchange.get_instruments()
    stock = all_instruments[underlying_stock_id]
    options = {instrument_id: instrument
               for instrument_id, instrument in all_instruments.items()
               if instrument.instrument_type == InstrumentType.STOCK_OPTION
               and instrument.base_instrument_id == underlying_stock_id}
    return options
def load_instruments_for_OB5X():
    all_instruments = exchange.get_instruments()
    filtered_dict = {key: value for key, value in all_instruments.items() if value.instrument_group == 'OB5X' and value.instrument_type == InstrumentType.INDEX_OPTION}
    return filtered_dict
def get_midpoint_value(instrument_id):
    """
    This function calculates the current midpoint of the order book supplied by the exchange for the instrument
    specified by <instrument_id>, returning None if either side or both sides do not have any orders available.
    """
    order_book = exchange.get_last_price_book(instrument_id=instrument_id)

    # If the instrument doesn't have prices at all or on either side, we cannot calculate a midpoint and return None
    if not (order_book and order_book.bids and order_book.asks):
        return None
    else:
        midpoint = (order_book.bids[0].price + order_book.asks[0].price) / 2.0
        return midpoint
def get_OB5X_value(side):
    """
    Calculates the value of the OB5X index
    """
    
    val = 0 
    if(side=='ask'):
        if(len(exchange.get_last_price_book('NVDA').asks)==0 or len(exchange.get_last_price_book('ING').asks)==0 or len(exchange.get_last_price_book('SAN').asks)==0 or len(exchange.get_last_price_book('PFE').asks)==0 or len(exchange.get_last_price_book('CSCO').asks) == 0):
            return None
        NVDA = exchange.get_last_price_book(instrument_id='NVDA').asks[0].price
        ING = exchange.get_last_price_book(instrument_id='ING').asks[0].price
        SAN = exchange.get_last_price_book(instrument_id='SAN').asks[0].price
        PFE = exchange.get_last_price_book(instrument_id='PFE').asks[0].price
        CSCO = exchange.get_last_price_book(instrument_id='CSCO').asks[0].price

        val = (908.06*NVDA + 129.24*ING + 124.78*SAN + 2245.39*PFE + 953.21*CSCO)/1000
    else:
        if(len(exchange.get_last_price_book('NVDA').bids)==0 or len(exchange.get_last_price_book('ING').bids)==0 or len(exchange.get_last_price_book('SAN').bids)==0 or len(exchange.get_last_price_book('PFE').bids)==0 or len(exchange.get_last_price_book('CSCO').bids) == 0):
            return None
        NVDA = exchange.get_last_price_book(instrument_id='NVDA').bids[0].price
        ING = exchange.get_last_price_book(instrument_id='ING').bids[0].price
        SAN = exchange.get_last_price_book(instrument_id='SAN').bids[0].price
        PFE = exchange.get_last_price_book(instrument_id='PFE').bids[0].price
        CSCO = exchange.get_last_price_book(instrument_id='CSCO').bids[0].price

        val = (908.06*NVDA + 129.24*ING + 124.78*SAN + 2245.39*PFE + 953.21*CSCO)/1000
    

    return val
def ETF_val(index_val):
    C = 2.5
    M = 0.25
    return C + M*index_val
def calculate_theoretical_future_value(underlying_value, expiry,  interest_rate=0.03):
    """
    This function calculates the current fair future value with: F = X exp(rt)

    expiry: dt.date          -  Expiry date of the future -  third Friday of current month, at 12:00 UTC
    underlying_value:        -  value of the underlying stock/index
    interest_rate:           -  interest rate of future
    """
    time_to_expiry = calculate_current_time_to_date(expiry)

    future_value = underlying_value*exp(interest_rate*time_to_expiry)

    return future_value
def calculate_theoretical_option_value(expiry, strike, option_kind, stock_value, interest_rate, volatility):
    """
    This function calculates the current fair call or put value based on Black & Scholes assumptions.

    expiry: dt.date          -  Expiry date of the option
    strike: float            -  Strike price of the option
    option_kind: OptionKind  -  Type of the option
    stock_value:             -  Assumed stock value when calculating the Black-Scholes value
    interest_rate:           -  Assumed interest rate when calculating the Black-Scholes value
    volatility:              -  Assumed volatility of when calculating the Black-Scholes value
    """
    time_to_expiry = calculate_current_time_to_date(expiry)


    if option_kind == OptionKind.CALL:
        option_value = call_value(S=stock_value, K=strike, T=time_to_expiry, r=interest_rate, sigma=volatility)
    elif option_kind == OptionKind.PUT:
        option_value = put_value(S=stock_value, K=strike, T=time_to_expiry, r=interest_rate, sigma=volatility)

    return option_value
def credit_calc (instrument_id, volume_steps, increment):
    position = exchange.get_positions()
    instrument_position = position[instrument_id]
    credit = -round(instrument_position / volume_steps) * increment
    return credit

In [39]:
while True:
    CSCO_ID = "CSCO"

    # CSCO
    # update CSCO quotes - max 4 exchange actions

    bid_price = get_CSCO_value('bid')
    ask_price = get_CSCO_value('ask')
    mid_point = (ask_price + bid_price)/2
    price_diff = (ask_price - bid_price)

    if(bid_price==None or ask_price==None):
        continue

    order_book = exchange.get_last_price_book("CSCO")
    print(f"best bid price at: {order_book.bids[0].price}")

    print(f"bid price at: {bid_price}")
    print(f"ask price at: {ask_price}")
    print(f"best ask price at: {order_book.asks[0].price}")
    print(f"mid point at: {mid_point}")
    print("~~~~~~~~~~~~#")
    #mid_point += credit_calc(CSCO_ID, 10, 0.1)
    update_quotes(instrument_id=CSCO_ID,
                theoretical_price = mid_point,
                credit = 0.20,
                volume = 53,
                position_limit = 100,
                tick_size = 0.10)
    time.sleep(3.0)

    # hedge_delta_position(SAN_ID, get_midpoint_value(SAN_ID), futures=None, options=None, dual=SAN_DUAL)

best bid price at: 36.9
bid price at: 36.300458450918484
ask price at: 36.78225469728602
best ask price at: 37.0
mid point at: 36.54135657410225
~~~~~~~~~~~~#
best bid price at: 36.9
bid price at: 36.8284470368544
ask price at: 36.79534520200167
best ask price at: 37.0
mid point at: 36.811896119428035
~~~~~~~~~~~~#
best bid price at: 37.1
bid price at: 37.05708710567452
ask price at: 37.2771414483692
best ask price at: 37.2
mid point at: 37.16711427702186
~~~~~~~~~~~~#
best bid price at: 37.1
bid price at: 37.27473064697182
ask price at: 37.3037903504999
best ask price at: 37.2
mid point at: 37.28926049873586
~~~~~~~~~~~~#
best bid price at: 37.1
bid price at: 37.82514136444226
ask price at: 37.887302902823095
best ask price at: 37.2
mid point at: 37.85622213363268
~~~~~~~~~~~~#
best bid price at: 37.0
bid price at: 37.72987799120866
ask price at: 38.00758909369394
best ask price at: 37.1
mid point at: 37.868733542451295
~~~~~~~~~~~~#
best bid price at: 37.1
bid price at: 37.7836436881

IndexError: list index out of range

2024-02-08 23:06:01,828 [client    ] [Thread-8 (_thread_entry_point)] Forcing a disconnect due to an error: Closing connection because someone else logged in with the same credentials. Only one session may be active at the same time.
